In [1]:
import pandas as pd 
import numpy as np

from tensorflow.python.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.python.keras.layers import Conv2D, Flatten, Dense,Dropout,GRU,Bidirectional
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.wrappers.scikit_learn import KerasClassifier
from keras.layers import Input
from tensorflow.keras.models import load_model
from tensorflow import keras


from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

import tensorflow as tf
tf.config.run_functions_eagerly(True)

Using TensorFlow backend.


###### Importing Pre-trained from pickle 

In [3]:
train_df = pd.read_pickle('train.pkl')
train_df.name = 'Train Dataset'
train_df.head()

,hmid,moment,social,agency
0,27674,i happy my son got marks his examination,1,0
1,27685,went movies my friends it fun,1,1
2,27691,hot kiss my girl friend last night made my day,1,1
3,27701,my son woke me fantastic breakfast eggs his sp...,1,0
4,27712,my older daughter keeps patting my younger dau...,1,0


In [4]:
test_df = pd.read_pickle('test.pkl')
test_df.name = "Test Dataset"
test_df.head()

,hmid,moment,social,agency
0,0,i able spend day my fiance shopping yesterday,0,1
1,1,i able play my cat,0,1
2,2,i able clean my room gold my laundry,0,1
3,3,i spend day party beach i happy see friends i ...,1,1
4,4,my cat greeting me i got home work she sweet c...,0,0


In [5]:
X_train = train_df['moment'].values
y_train_social = train_df['social'].values
y_train_agency = train_df['agency'].values

In [6]:
X_test = test_df['moment'].values
y_test_social = test_df['social'].values
y_test_agency = test_df['agency'].values

In [7]:
def transformResults(predictions):
    
    predictions = predictions[:, 0]
    print(f'First 5 values before conversion : {predictions[:5]}')
    
    predictions = [1 if val>0.5 else 0 for val in predictions]
    print(f'First 5 values after the conversion : {predictions[:5]}')
    return predictions

###### function to get all required metrics for our models

In [8]:
# craeting lists to store results
model_name =[]
social_accuracy=[]
agency_accuracy=[]

###### Tokenization and vectorizing the dataset using spacy package

In [9]:
import spacy


In [10]:
def get_spacy_vectors(nlp, dataset):
    vectors = []
    cnt = 0

    for data_row in dataset:

        data_block = ""
        text_block = data_row[0]

        for i in range(0, len(data_row)):
            
            if('|' in str(data_row[i])):
                for tag in data_row[i].split('|'):
                    data_block = data_block + " " + str(tag)
        
            else:
                data_block = data_block + " " + str(data_row[i])

        # print(data_block)
        document = nlp(data_block)
        vectors.append(document.vector)
        # print(type(document.vector))

        cnt += 1
        if(cnt%10 == 0):
            print(cnt,"/",len(dataset))
    return np.array(vectors)

###### downloading Language Model:
    
    The model (en_core_web_sm) is the largest English model of spaCy with size 788 MB. There are smaller models in English and some other models for other languages (English, German, French, Spanish, Portuguese, Italian, Dutch, Greek).

In [11]:
nlp = spacy.load("en_core_web_sm")

X_train = get_spacy_vectors(nlp, X_train)
X_test = get_spacy_vectors(nlp, X_test)

10 / 9921
20 / 9921
30 / 9921
40 / 9921
50 / 9921
60 / 9921
70 / 9921
80 / 9921
90 / 9921
100 / 9921
110 / 9921
120 / 9921
130 / 9921
140 / 9921
150 / 9921
160 / 9921
170 / 9921
180 / 9921
190 / 9921
200 / 9921
210 / 9921
220 / 9921
230 / 9921
240 / 9921
250 / 9921
260 / 9921
270 / 9921
280 / 9921
290 / 9921
300 / 9921
310 / 9921
320 / 9921
330 / 9921
340 / 9921
350 / 9921
360 / 9921
370 / 9921
380 / 9921
390 / 9921
400 / 9921
410 / 9921
420 / 9921
430 / 9921
440 / 9921
450 / 9921
460 / 9921
470 / 9921
480 / 9921
490 / 9921
500 / 9921
510 / 9921
520 / 9921
530 / 9921
540 / 9921
550 / 9921
560 / 9921
570 / 9921
580 / 9921
590 / 9921
600 / 9921
610 / 9921
620 / 9921
630 / 9921
640 / 9921
650 / 9921
660 / 9921
670 / 9921
680 / 9921
690 / 9921
700 / 9921
710 / 9921
720 / 9921
730 / 9921
740 / 9921
750 / 9921
760 / 9921
770 / 9921
780 / 9921
790 / 9921
800 / 9921
810 / 9921
820 / 9921
830 / 9921
840 / 9921
850 / 9921
860 / 9921
870 / 9921
880 / 9921
890 / 9921
900 / 9921
910 / 9921
920 / 99

6930 / 9921
6940 / 9921
6950 / 9921
6960 / 9921
6970 / 9921
6980 / 9921
6990 / 9921
7000 / 9921
7010 / 9921
7020 / 9921
7030 / 9921
7040 / 9921
7050 / 9921
7060 / 9921
7070 / 9921
7080 / 9921
7090 / 9921
7100 / 9921
7110 / 9921
7120 / 9921
7130 / 9921
7140 / 9921
7150 / 9921
7160 / 9921
7170 / 9921
7180 / 9921
7190 / 9921
7200 / 9921
7210 / 9921
7220 / 9921
7230 / 9921
7240 / 9921
7250 / 9921
7260 / 9921
7270 / 9921
7280 / 9921
7290 / 9921
7300 / 9921
7310 / 9921
7320 / 9921
7330 / 9921
7340 / 9921
7350 / 9921
7360 / 9921
7370 / 9921
7380 / 9921
7390 / 9921
7400 / 9921
7410 / 9921
7420 / 9921
7430 / 9921
7440 / 9921
7450 / 9921
7460 / 9921
7470 / 9921
7480 / 9921
7490 / 9921
7500 / 9921
7510 / 9921
7520 / 9921
7530 / 9921
7540 / 9921
7550 / 9921
7560 / 9921
7570 / 9921
7580 / 9921
7590 / 9921
7600 / 9921
7610 / 9921
7620 / 9921
7630 / 9921
7640 / 9921
7650 / 9921
7660 / 9921
7670 / 9921
7680 / 9921
7690 / 9921
7700 / 9921
7710 / 9921
7720 / 9921
7730 / 9921
7740 / 9921
7750 / 9921
7760

3640 / 17215
3650 / 17215
3660 / 17215
3670 / 17215
3680 / 17215
3690 / 17215
3700 / 17215
3710 / 17215
3720 / 17215
3730 / 17215
3740 / 17215
3750 / 17215
3760 / 17215
3770 / 17215
3780 / 17215
3790 / 17215
3800 / 17215
3810 / 17215
3820 / 17215
3830 / 17215
3840 / 17215
3850 / 17215
3860 / 17215
3870 / 17215
3880 / 17215
3890 / 17215
3900 / 17215
3910 / 17215
3920 / 17215
3930 / 17215
3940 / 17215
3950 / 17215
3960 / 17215
3970 / 17215
3980 / 17215
3990 / 17215
4000 / 17215
4010 / 17215
4020 / 17215
4030 / 17215
4040 / 17215
4050 / 17215
4060 / 17215
4070 / 17215
4080 / 17215
4090 / 17215
4100 / 17215
4110 / 17215
4120 / 17215
4130 / 17215
4140 / 17215
4150 / 17215
4160 / 17215
4170 / 17215
4180 / 17215
4190 / 17215
4200 / 17215
4210 / 17215
4220 / 17215
4230 / 17215
4240 / 17215
4250 / 17215
4260 / 17215
4270 / 17215
4280 / 17215
4290 / 17215
4300 / 17215
4310 / 17215
4320 / 17215
4330 / 17215
4340 / 17215
4350 / 17215
4360 / 17215
4370 / 17215
4380 / 17215
4390 / 17215
4400 / 17215

9950 / 17215
9960 / 17215
9970 / 17215
9980 / 17215
9990 / 17215
10000 / 17215
10010 / 17215
10020 / 17215
10030 / 17215
10040 / 17215
10050 / 17215
10060 / 17215
10070 / 17215
10080 / 17215
10090 / 17215
10100 / 17215
10110 / 17215
10120 / 17215
10130 / 17215
10140 / 17215
10150 / 17215
10160 / 17215
10170 / 17215
10180 / 17215
10190 / 17215
10200 / 17215
10210 / 17215
10220 / 17215
10230 / 17215
10240 / 17215
10250 / 17215
10260 / 17215
10270 / 17215
10280 / 17215
10290 / 17215
10300 / 17215
10310 / 17215
10320 / 17215
10330 / 17215
10340 / 17215
10350 / 17215
10360 / 17215
10370 / 17215
10380 / 17215
10390 / 17215
10400 / 17215
10410 / 17215
10420 / 17215
10430 / 17215
10440 / 17215
10450 / 17215
10460 / 17215
10470 / 17215
10480 / 17215
10490 / 17215
10500 / 17215
10510 / 17215
10520 / 17215
10530 / 17215
10540 / 17215
10550 / 17215
10560 / 17215
10570 / 17215
10580 / 17215
10590 / 17215
10600 / 17215
10610 / 17215
10620 / 17215
10630 / 17215
10640 / 17215
10650 / 17215
10660 / 172

15810 / 17215
15820 / 17215
15830 / 17215
15840 / 17215
15850 / 17215
15860 / 17215
15870 / 17215
15880 / 17215
15890 / 17215
15900 / 17215
15910 / 17215
15920 / 17215
15930 / 17215
15940 / 17215
15950 / 17215
15960 / 17215
15970 / 17215
15980 / 17215
15990 / 17215
16000 / 17215
16010 / 17215
16020 / 17215
16030 / 17215
16040 / 17215
16050 / 17215
16060 / 17215
16070 / 17215
16080 / 17215
16090 / 17215
16100 / 17215
16110 / 17215
16120 / 17215
16130 / 17215
16140 / 17215
16150 / 17215
16160 / 17215
16170 / 17215
16180 / 17215
16190 / 17215
16200 / 17215
16210 / 17215
16220 / 17215
16230 / 17215
16240 / 17215
16250 / 17215
16260 / 17215
16270 / 17215
16280 / 17215
16290 / 17215
16300 / 17215
16310 / 17215
16320 / 17215
16330 / 17215
16340 / 17215
16350 / 17215
16360 / 17215
16370 / 17215
16380 / 17215
16390 / 17215
16400 / 17215
16410 / 17215
16420 / 17215
16430 / 17215
16440 / 17215
16450 / 17215
16460 / 17215
16470 / 17215
16480 / 17215
16490 / 17215
16500 / 17215
16510 / 17215
16520 

In [12]:
X_train=np.asarray(X_train)
X_test=np.asarray(X_test)

X_train.shape,X_test.shape

((9921, 96), (17215, 96))

In [13]:
def sequencial_model():
    classifier = Sequential()

    classifier.add(Dense(40,
                         activation = 'relu',
                         input_dim = X_train.shape[1]))

    classifier.add(Dense( 20,
                     activation = 'sigmoid'))

    classifier.add(Dense(1,
                     activation = 'sigmoid'))

    classifier.compile(optimizer='adam',
                   loss='binary_crossentropy',
                   metrics=['accuracy'])
    classifier.summary()
    
    return classifier

model_name.append("sequencial_model")

In [14]:
model = sequencial_model()
print("\n \n ----------------------------------- ---------------------------------------------- \n \n")
model.fit(X_train, y_train_social,
                    epochs=5,
                    verbose=True,
                    validation_split=0.2,
                batch_size=10)
predicted = model.predict_classes(X_test)

print("\n \n ----------------------------------- ---------------------------------------------- \n \n")

print(classification_report(y_test_social, predicted))


accuracy_s= accuracy_score(y_test_social, predicted)

social_accuracy.append("accuracy")

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 40)                3880      
_________________________________________________________________
dense_1 (Dense)              (None, 20)                820       
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 21        
Total params: 4,721
Trainable params: 4,721
Non-trainable params: 0
_________________________________________________________________

 
 ----------------------------------- ---------------------------------------------- 
 

Epoch 1/5
794/794 [==============================] - 17s 21ms/step - loss: 0.6654 - accuracy: 0.5968 - val_loss: 0.5937 - val_accuracy: 0.6947
Epoch 2/5
794/794 [==============================] - 13s 16ms/step - loss: 0.5860 - accuracy: 0.6953 - val_loss: 0.5545 - val_accuracy: 0.7229
Epoch 3

In [15]:
model = sequencial_model()

print("\n \n ----------------------------------- ---------------------------------------------- \n \n")

model.fit(X_train, y_train_agency,
                    epochs=5,
                    verbose=True,
                    validation_split=0.2,
                    batch_size=10)
predicted = model.predict_classes(X_test)

print("\n \n ----------------------------------- ---------------------------------------------- \n \n")

print(classification_report(y_test_agency, predicted))

accuracy_s= accuracy_score(y_test_agency, predicted)

agency_accuracy.append("accuracy")

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 40)                3880      
_________________________________________________________________
dense_4 (Dense)              (None, 20)                820       
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 21        
Total params: 4,721
Trainable params: 4,721
Non-trainable params: 0
_________________________________________________________________

 
 ----------------------------------- ---------------------------------------------- 
 

Epoch 1/5
794/794 [==============================] - 15s 19ms/step - loss: 0.5812 - accuracy: 0.7312 - val_loss: 0.5336 - val_accuracy: 0.7567
Epoch 2/5
794/794 [==============================] - 11s 14ms/step - loss: 0.5234 - accuracy: 0.7524 - val_loss: 0.5049 - val_accuracy: 0.7768
Epoch

###### Deep neural networks Model for text classification

In [16]:
def Build_Model_DNN_Text(shape, nClasses, dropout=0.5):
   
    model = Sequential()
    node = 512 # number of nodes
    nLayers = 4 # number of  hidden layer

    model.add(Dense(node,input_dim=shape,activation='relu'))
    model.add(Dropout(dropout))
    for i in range(0,nLayers):
        model.add(Dense(node,input_dim=node,activation='relu'))
        model.add(Dropout(dropout))
    model.add(Dense(nClasses, activation='relu'))

    model.compile(loss='sparse_categorical_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])
    model.summary()

    return model

In [17]:
model_DNN = Build_Model_DNN_Text(X_train.shape[1],20)

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              (None, 512)               49664     
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_7 (Dense)              (None, 512)               262656    
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_8 (Dense)              (None, 512)               262656    
_________________________________________________________________
dropout_2 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_9 (Dense)              (None, 512)              

In [18]:
model_DNN.fit(X_train, y_train_social,
                    epochs=5,
                    verbose=True,
                    validation_split=0.2,
                batch_size=10)
predicted = model_DNN.predict_classes(X_test)

print("\n \n ----------------------------------- ---------------------------------------------- \n \n")

print(classification_report(y_test_social, predicted))


accuracy_s= accuracy_score(y_test_social, predicted)

social_accuracy.append("accuracy")

Epoch 1/5
794/794 [==============================] - 31s 38ms/step - loss: 1.0056 - accuracy: 0.4829 - val_loss: 0.6931 - val_accuracy: 0.4957
Epoch 2/5
794/794 [==============================] - 33s 41ms/step - loss: 0.6932 - accuracy: 0.4684 - val_loss: 0.6931 - val_accuracy: 0.5043
Epoch 3/5
794/794 [==============================] - 37s 47ms/step - loss: 0.6934 - accuracy: 0.4761 - val_loss: 0.6931 - val_accuracy: 0.5043
Epoch 4/5
794/794 [==============================] - 33s 41ms/step - loss: 0.6932 - accuracy: 0.5414 - val_loss: 0.6931 - val_accuracy: 0.5043
Epoch 5/5
794/794 [==============================] - 30s 38ms/step - loss: 0.6932 - accuracy: 0.5402 - val_loss: 0.6931 - val_accuracy: 0.5043

 
 ----------------------------------- ---------------------------------------------- 
 

              precision    recall  f1-score   support

           0       0.00      0.00      0.00      7417
           1       0.57      1.00      0.73      9798

    accuracy                  

In [19]:
model_DNN.fit(X_train, y_train_agency,
                    epochs=5,
                    verbose=True,
                    validation_split=0.2,
                batch_size=10)
predicted = model_DNN.predict_classes(X_test)

print("\n \n ----------------------------------- ---------------------------------------------- \n \n")

print(classification_report(y_test_agency, predicted))


accuracy_s= accuracy_score(y_test_agency, predicted)

agency_accuracy.append("accuracy")

Epoch 1/5
794/794 [==============================] - 30s 37ms/step - loss: 0.6932 - accuracy: 0.7259 - val_loss: 0.6931 - val_accuracy: 0.7340
Epoch 2/5
794/794 [==============================] - 28s 35ms/step - loss: 0.6931 - accuracy: 0.7258 - val_loss: 0.6931 - val_accuracy: 0.7340
Epoch 3/5
794/794 [==============================] - 28s 35ms/step - loss: 0.6931 - accuracy: 0.7257 - val_loss: 0.6931 - val_accuracy: 0.7340
Epoch 4/5
794/794 [==============================] - 30s 37ms/step - loss: 0.6931 - accuracy: 0.7259 - val_loss: 0.6931 - val_accuracy: 0.7340
Epoch 5/5
794/794 [==============================] - 32s 40ms/step - loss: 0.6931 - accuracy: 0.7258 - val_loss: 0.6931 - val_accuracy: 0.7340

 
 ----------------------------------- ---------------------------------------------- 
 

              precision    recall  f1-score   support

           0       0.00      0.00      0.00      5059
           1       0.71      1.00      0.83     12156

    accuracy                  